In [5]:
import spiceypy
import datetime

# get today's date and ensure it takes midnight for a time
today_date = datetime.datetime.today().strftime("%Y-%m-%dT00:00:00")


# get the needed kernels
spiceypy.furnsh("kernels/lsk/naif0012.tls")
spiceypy.furnsh("kernels/spk/de432s.bsp")

# put the date into SPICE for computation
et_midnight = spiceypy.utc2et(today_date)

# we get back the earth's vectors x, y and z coordinates as well as the light time, velocities in km/s
earth_vectors, earth_light_time = spiceypy.spkgeo(targ=399, et=et_midnight, ref="ECLIPJ2000", obs=10)
print(earth_vectors)

[-1.26019095e+08  7.72745421e+07 -3.91439090e+03 -1.60452363e+01
 -2.55098727e+01  8.87659649e-04]


In [6]:
import numpy as np

#convert the output into an array for easier accessing
earth_vectors = np.array(earth_vectors)

# recalculate distance to the sun to check if np does the same as the math from comp_earth
earth_sun_distance = np.linalg.norm(earth_vectors[:3])

# calculate the speed at which the earth goes around the sun
earth_orbit_speed = np.linalg.norm(earth_vectors[3:])

print(earth_orbit_speed)

30.13641009463656


In [7]:
spiceypy.furnsh("kernels/pck/gm_de431.tpc")

# we don't need the first parameter, but we do want the gravitational parameter x the mass of the sun
# the function needs the terrestal body id, the Sun so 10, what we want back(the gravitational parameter x the mass) and the amount of dimensions it has
_, GM_SUN = spiceypy.bodvcd(bodyid=10, item="GM", maxn=1)

# gives back the velocity in km/s
velocity_orbit_func = lambda gm, r: np.sqrt(gm/r)

# this determines the speed by using the function above with the theoretically correct data to compare
earth_orbit_speed_theoretical = velocity_orbit_func(GM_SUN[0], earth_sun_distance)

print(earth_orbit_speed_theoretical)

29.962785717358518


In [9]:
earth_vectors = earth_vectors[:3]

# normalize the vector
earth_position_sun_normed = earth_vectors / earth_sun_distance

# position during autumn
earth_position_sun_autmn = np.array([1.0, 0.0, 0.0])

# calculate the angle of the earth in degrees from autumn until now
angle_distance_degrees = np.degrees(np.arccos(np.dot(earth_position_sun_normed, earth_position_sun_autmn)))

print(angle_distance_degrees)

148.48349552213173
